In [0]:
%sh
pip install editdistance
pip install minisom

Requirement already satisfied: editdistance in /databricks/python3/lib/python3.7/site-packages (0.5.3)
Requirement already satisfied: minisom in /databricks/python3/lib/python3.7/site-packages (2.2.6)

In [0]:
import traceback
import logging
import datetime
from datetime import date
import time
import hashlib
from shutil import copyfile
import json
import os
import sys
sys.path.insert(0, os.path.abspath("/dbfs/mnt/mountdatalake/AZ_schemaMatchcodebase"))
import SchemaMatchEngine
import UIMeasures
import SchemaPickleDump
from shutil import copyfile

file_date = (str(datetime.datetime.now())).replace(" ","")
local_log_filename= "/tmp/schema-Logfile-"+file_date+".log"
log_filename= "/dbfs/mnt/mountdatalake/AZ_schemaData/log/"+"schema-Logfile-"+file_date+".log"

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler(local_log_filename)
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.addHandler(stream_handler)
logging.getLogger("py4j").setLevel(logging.ERROR)

errFlag=False
try:
  TestinputSource=dbutils.widgets.get('test')
  #TestinputSource="SP_Main.csv" #For Local run
  logger.info("Getting test file details from UI: "+str(TestinputSource))
  TraininputSource=dbutils.widgets.get('train')
  #TraininputSource="HP_Main.csv"# For local run
  logger.info("Getting training file details from UI: "+str(TraininputSource))
  testfileURL, trainPickleF, testPickleF = SchemaPickleDump.process(TestinputSource,TraininputSource)
  resultFile = SchemaMatchEngine.processEngine(trainPickleF,testPickleF)
  logger.info("Schema matching process completed and result file has been generated ")
  dataUI = UIMeasures.measureProcess(testfileURL,resultFile)
  logger.info("UI Measure activity Completed ")
except Exception as e:
  errFlag=True
  logger.error("Exception occured details are : "+str(e))
finally:
  if errFlag==False:
    logger.info("Successfully completed the Schema Matching Operation")
    copyfile(local_log_filename,log_filename)
    dbutils.notebook.exit(json.dumps({"filename":str(dataUI)}))
  else:
    logger.info("Schema Matching Operation not successful")
    copyfile(local_log_filename,log_filename)
    dbutils.notebook.exit(json.dumps({"status": "ERROR"}))

{"filename": "SP_Main_UIMeasures.csv"}